# Interactive Visualization with Folium

This notebook demonstrates how to create interactive maps to visualize remote sensing data using the `folium` library in Python. It displays raster data (e.g., Sentinel-2 or Landsat RGB composites) and vector data (e.g., area of interest boundaries) on an interactive map, enabling dynamic exploration of geospatial data.

## Prerequisites
- Install required libraries: `folium`, `rasterio`, `geopandas`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- A GeoTIFF file (e.g., `sentinel_rgb.tif` or `landsat_rgb.tif`) from a previous notebook (e.g., `21_download_data.ipynb`).
- A GeoJSON file defining the area of interest (e.g., `aoi.geojson`). Replace file paths with your own data.

## Learning Objectives
- Create an interactive map with Folium to display raster imagery.
- Overlay vector data (e.g., AOI boundaries) on the map.
- Save the interactive map as an HTML file for sharing.

In [ ]:
# Import required libraries
import folium
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from folium.raster_layers import ImageOverlay
import os

## Step 1: Load Raster and Vector Data

Load the RGB composite GeoTIFF and the area of interest (AOI) GeoJSON file.

In [ ]:
# Define file paths
raster_path = 'remote_sensing_data/sentinel_rgb.tif'  # Replace with your RGB GeoTIFF (e.g., from notebook 21)
aoi_path = 'aoi.geojson'  # Path to your GeoJSON file

# Load raster
with rasterio.open(raster_path) as src:
    raster_data = src.read().transpose(1, 2, 0)  # Shape: (height, width, bands)
    bounds = src.bounds  # Get geographic bounds
    crs = src.crs

# Load AOI
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != crs:
    aoi_gdf = aoi_gdf.to_crs(crs)

# Print basic information
print(f'Raster shape: {raster_data.shape}')
print(f'Raster CRS: {crs}')
print(f'AOI CRS: {aoi_gdf.crs}')

## Step 2: Prepare Raster for Folium

Normalize the RGB raster data and convert it to a format suitable for Folium.

In [ ]:
# Normalize raster for visualization
raster_normalized = raster_data / np.percentile(raster_data, 98) if np.percentile(raster_data, 98) > 0 else raster_data
raster_normalized = np.clip(raster_normalized, 0, 1)

# Convert bounds to Folium format (lat, lon)
from rasterio.warp import transform_bounds
bounds_latlon = transform_bounds(crs, 'EPSG:4326', *bounds)
folium_bounds = [[bounds_latlon[1], bounds_latlon[0]], [bounds_latlon[3], bounds_latlon[2]]]

# Save normalized raster as a temporary PNG for Folium
import imageio
temp_png = 'temp_raster.png'
imageio.imwrite(temp_png, (raster_normalized * 255).astype(np.uint8))

## Step 3: Create Interactive Folium Map

Create a Folium map centered on the AOI, overlay the raster, and add the AOI boundary.

In [ ]:
# Calculate map center
center_lat = (bounds_latlon[1] + bounds_latlon[3]) / 2
center_lon = (bounds_latlon[0] + bounds_latlon[2]) / 2

# Initialize Folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

# Add raster as ImageOverlay
ImageOverlay(
    image=temp_png,
    bounds=folium_bounds,
    opacity=0.8
).add_to(m)

# Add AOI boundary
folium.GeoJson(
    aoi_gdf.to_crs('EPSG:4326').to_json(),
    name='AOI',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0}
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display map
m

## Step 4: Save the Interactive Map

Save the Folium map as an HTML file for sharing or viewing in a web browser.

In [ ]:
# Save map to HTML
output_map_path = 'remote_sensing_map.html'
m.save(output_map_path)

print(f'Interactive map saved to: {output_map_path}')

# Clean up temporary PNG
os.remove(temp_png)

## Step 5: Static Visualization for Reference

Create a static visualization of the RGB composite with the AOI boundary for reference.

In [ ]:
# Plot static RGB composite with AOI
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(raster_normalized)
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
plt.title('Static RGB Composite with AOI')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `sentinel_rgb.tif` with your own RGB GeoTIFF (e.g., from `21_download_data.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Customize the Folium map with additional layers (e.g., other rasters or vector data).
- Explore advanced Folium features like popups or custom tiles.
- Revisit previous notebooks (e.g., `21_download_data.ipynb`) for data preprocessing or proceed with analysis tasks like classification or change detection.

## Notes
- Ensure the raster and AOI have compatible CRS; Folium requires data in EPSG:4326 for display.
- Large rasters may need downsampling to avoid memory issues in Folium.
- See `docs/installation.md` for troubleshooting library installation.